In [1]:
import os
import pandas as pd


###############################################################################################
#            CONVERTE O ARQUIVO PARA .CSV E SALVA EM OUTRA PASTA O ARQUIVO ORIGINAL           #
###############################################################################################
def xlsx_to_csv(input_file, output_file, processed_folder):
    # Carregue o arquivo Excel usando pandas
    df = pd.read_excel(input_file)

    # Salve o DataFrame como um arquivo CSV separado por ponto e vírgula
    df.to_csv(output_file, index=False, sep=';')

    print(f'O arquivo {input_file} foi convertido para {output_file}.')

    # Mova o arquivo original para a pasta de processados com '_processado' no nome
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    # Construa o caminho completo para o arquivo original na pasta de processados
    processed_original_file_path = os.path.join(processed_folder, os.path.basename(input_file.replace('.xlsx', '_processado.xlsx')))

    # Mova o arquivo original para a pasta de processados
    os.rename(input_file, processed_original_file_path)

    print(f'O arquivo original foi movido para a pasta {processed_folder}.')

def convert_files_in_folder(folder_path, processed_folder):
    # Liste todos os arquivos na pasta de origem
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

    for file in files:
        # Construa o caminho completo para o arquivo de entrada
        input_file_path = os.path.join(folder_path, file)

        # Construa o caminho completo para o arquivo de saída
        output_file_path = os.path.join(source_folder, file.replace('.xlsx', '.csv'))

        # Chame a função para realizar a conversão e mover o arquivo
        xlsx_to_csv(input_file_path, output_file_path, processed_folder)

# Substitua com o caminho da sua pasta de origem e a pasta de processados
source_folder = r'C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos'
processed_folder = r'C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/processados'

# Chame a função para converter todos os arquivos na pasta de origem
convert_files_in_folder(source_folder, processed_folder)



O arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base 2017.xlsx foi convertido para C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base 2017.csv.
O arquivo original foi movido para a pasta C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/processados.
O arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2018.xlsx foi convertido para C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2018.csv.
O arquivo original foi movido para a pasta C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/processados.
O arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2019.xlsx foi convertido para C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2019.csv.
O arquivo original foi movido para a pasta C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/processados.


In [12]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

# Caminho para a chave do IAM
key_path = "C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/access/GB.json"

# Carregar credenciais
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

def load_to_bigquery(file_path, dataset_id, table_id, credentials):
    # Carregar dados com pandas
    df = pd.read_csv(file_path, delimiter=';')

    # Inicializar cliente BigQuery
    client = bigquery.Client(credentials=credentials)

    # Criar referência para a tabela de destino
    table_ref = client.dataset(dataset_id).table(table_id)

    # Configurações do job de carregamento
    job_config = bigquery.LoadJobConfig(
        skip_leading_rows=0,
        source_format=bigquery.SourceFormat.CSV,
    )

    # Carregar dados na tabela
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # Aguardar término do job

    print(f'Dados carregados na tabela {table_id} no BigQuery.')

    # Imprimir informações sobre o arquivo
    print(f'Nome do arquivo carregado: {file_path}')
    print(f'Número de linhas carregadas: {len(df)}')

    # Excluir o arquivo do diretório
    os.remove(file_path)
    print(f'Arquivo {file_path} excluído.')

# Diretório de origem
source_folder = r'C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos'

# Substitua com as informações do seu projeto, dataset e tabela no BigQuery
dataset_id = 'transient'
table_id = 'stg_base_vendas'

# Iterar sobre os arquivos no diretório de origem
files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

for file in files:
    # Caminho completo para o arquivo de entrada
    input_file_path = os.path.join(source_folder, file)

    # Carregar dados para o BigQuery
    load_to_bigquery(input_file_path, dataset_id, table_id, credentials)


Dados carregados na tabela stg_base_vendas no BigQuery.
Nome do arquivo carregado: C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base 2017.csv
Número de linhas carregadas: 1000
Arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base 2017.csv excluído.
Dados carregados na tabela stg_base_vendas no BigQuery.
Nome do arquivo carregado: C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2018.csv
Número de linhas carregadas: 1000
Arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2018.csv excluído.
Dados carregados na tabela stg_base_vendas no BigQuery.
Nome do arquivo carregado: C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2019.csv
Número de linhas carregadas: 1000
Arquivo C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/arquivos\Base_2019.csv excluído.
